In [1]:
import pandas as pd
import numpy as np
import os

In [7]:
path = "data"
df_i = pd.read_pickle(os.path.join(path, "cleaned_df_i.pkl"))
df_c_train = pd.read_pickle(os.path.join(path, "df_c_train.pkl"))
df_t_train = pd.read_pickle(os.path.join(path, "df_t_train.pkl"))

In [4]:
df_c_train.head()

,customer_id,age,fashion_news_frequency
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,49.0,NONE
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,25.0,NONE
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,54.0,NONE
6,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,20.0,NONE
13,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,56.0,Regularly


In [5]:
df_t_train.head()

,t_dat,customer_id,article_id,price,sales_channel_id
8113895,2019-04-01,000747860042b94e85707605c2a627c6ba30c4117d025d...,732725002,0.050831,1
8113896,2019-04-01,001021122bdb55ec4235f35eb50164c268ec7c4bcfff90...,697060014,0.025407,2
8113897,2019-04-01,001021122bdb55ec4235f35eb50164c268ec7c4bcfff90...,758200004,0.067780,2
8113898,2019-04-01,0021ce2be745fe55f9b2bc784dbfd0e374c315ccd15aab...,769965001,0.042356,2
8113899,2019-04-01,0021ce2be745fe55f9b2bc784dbfd0e374c315ccd15aab...,709269004,0.042356,2


In [23]:
df_t_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3738625 entries, 8113895 to 11852519
Data columns (total 5 columns):
 #   Column            Dtype         
---  ------            -----         
 0   t_dat             datetime64[ns]
 1   customer_id       object        
 2   article_id        int64         
 3   price             float64       
 4   sales_channel_id  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 171.1+ MB


In [8]:
df_i.head()

,article_id,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_name,colour_group_name,perceived_colour_value_name,perceived_colour_master_name,index_group_name,garment_group_name,detail_desc
0,108775015,Strap top,253,Vest top,Garment Upper body,Solid,Black,Dark,Black,Ladieswear,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,Strap top,253,Vest top,Garment Upper body,Solid,White,Light,White,Ladieswear,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,Strap top (1),253,Vest top,Garment Upper body,Stripe,Off White,Dusty Light,White,Ladieswear,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,OP T-shirt (Idro),306,Bra,Underwear,Solid,Black,Dark,Black,Ladieswear,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,OP T-shirt (Idro),306,Bra,Underwear,Solid,White,Light,White,Ladieswear,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


# Content-Based Filtering

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
item_features = ['product_type_name', 'graphical_appearance_name', 'colour_group_name', 'section_name']

def getDesc(index):
    row = df_i.iloc[index, :]
    description = ""
    for col in item_features:
        description += " " + row[col]
    return description

items_desc = [getDesc(idx) if item is None else item for idx, item in enumerate(df_i['detail_desc'])]

In [11]:
vectoriser = TfidfVectorizer(ngram_range =(1,3),
                             strip_accents='unicode',
                             stop_words = 'english',
                             analyzer = 'word')

tfidf_matrix = vectoriser.fit_transform(items_desc)

In [24]:
class UserProfile:
    def __init__(self, data):
        self.interacted = {} # maps a user to a list of their purchased items
        self.time_interacted = {} # maps a user to a list of timestamps of their purchases
        self.data = data

    def get_user_interactions(self):
        user_groups = self.data.groupby('customer_id')
        self.interacted = user_groups['article_id'].apply(list).to_dict()
        self.time_interacted = user_groups['t_dat'].apply(list).to_dict()

train_userprofile = UserProfile(df_t_train)
train_userprofile.get_user_interactions()

In [25]:
sample_interaction = train_userprofile.interacted['002cb6b0ee98f590c120fb5f3c458a6588287db25901341cdb0f5e5a8cf8edcb']
sample_timestamp = train_userprofile.time_interacted['002cb6b0ee98f590c120fb5f3c458a6588287db25901341cdb0f5e5a8cf8edcb']
print(sample_interaction)
print(sample_timestamp)

[712914003, 777045001, 766643001]
[Timestamp('2019-05-01 00:00:00'), Timestamp('2019-05-01 00:00:00'), Timestamp('2019-05-01 00:00:00')]


In [70]:
sample_item_desc = []

for item in sample_interaction:
    idx = df_i[df_i['article_id'] == item].index[0]
    sample_item_desc.append(idx)

df_i.iloc[sample_item_desc, :]

,article_id,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_name,colour_group_name,perceived_colour_value_name,perceived_colour_master_name,index_group_name,garment_group_name,detail_desc
56079,712914003,Bond lace top,254,Top,Garment Upper body,Lace,White,Light,White,Ladieswear,Jersey Fancy,Short-sleeved lace top with a round neckline a...
74497,777045001,Haze shirt dress,265,Dress,Garment Full body,Solid,Light Beige,Dusty Light,White,Ladieswear,Blouses,Calf-length dress in viscose twill with a gran...
72107,766643001,Skat Blouse,258,Blouse,Garment Upper body,Solid,Light Yellow,Light,Yellow,Ladieswear,Blouses,"Short, straight-cut blouse in a linen weave wi..."


In [40]:
def calculate_exponential_weight(time_interacted, time_reference, constant):
    return np.exp(-1 * constant * (time_reference - time_interacted).days)

# Testing with constant = 0.05
calculate_exponential_weight(sample_timestamp[1], df_t_train['t_dat'].max(), 0.05)

0.11080315836233387

In [77]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

weights = []
scores = []

# For each interacted item by the user
for i in range(0, len(sample_interaction)):
    # Compute the weight to be applied
    weights.append(calculate_exponential_weight(sample_timestamp[i], df_t_train['t_dat'].max(), 0.05)) 

    # Perform search for similar products
    idx = df_i[df_i['article_id'] == sample_interaction[i]].index[0]
    query = [items_desc[idx]]
    query = vectoriser.transform(query)
    arr = cosine_similarity(query, tfidf_matrix).flatten()

    # assign the equivalent item score to be 0 (prevent interacted items from being recommended again)
    arr[arr >= 1.0] = 0 
    scores.append(arr)

# apply weight to each corresponding score
final_ranking = np.dot(weights, scores)

# lets try generating 20 recommendations
recommend_list = final_ranking.argsort()[::-1][:20]

recommend_list

df_i.iloc[recommend_list, :]


,article_id,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_name,colour_group_name,perceived_colour_value_name,perceived_colour_master_name,index_group_name,garment_group_name,detail_desc
76154,783832003,Skat Blouse,258,Blouse,Garment Upper body,All over pattern,White,Light,White,Ladieswear,Blouses,"Short, straight-cut blouse in a linen weave wi..."
83867,813636001,Blue bell top,254,Top,Garment Upper body,Lace,White,Light,White,Ladieswear,Jersey Fancy,Short-sleeved lace top with an opening at the ...
103402,911623001,Amiga lace dress,265,Dress,Garment Full body,Lace,Dark Blue,Dark,Blue,Ladieswear,Jersey Fancy,"Short, fitted dress in lace with a round neckl..."
58769,721135003,SPEED Snowdrop Top GCD,254,Top,Garment Upper body,Lace,Light Beige,Dusty Light,Beige,Ladieswear,Jersey Fancy,"Sleeveless lace top with a round neckline, con..."
58768,721135002,SPEED Snowdrop Top GCD,254,Top,Garment Upper body,Lace,White,Light,White,Ladieswear,Jersey Fancy,"Sleeveless lace top with a round neckline, con..."
93659,859152004,Buzz dress,265,Dress,Garment Full body,Solid,Black,Dark,Black,Ladieswear,Blouses,Long dress in viscose twill with a grandad col...
93658,859152001,Buzz dress,265,Dress,Garment Full body,All over pattern,Off White,Dusty Light,White,Ladieswear,Blouses,Long dress in viscose twill with a grandad col...
61283,732805002,KATE BLOUSE,258,Blouse,Garment Upper body,All over pattern,White,Light,White,Divided,Blouses,Blouse in a patterned viscose weave with a V-n...
61282,732805001,KATE BLOUSE,258,Blouse,Garment Upper body,All over pattern,Black,Dark,Black,Divided,Blouses,Blouse in a patterned viscose weave with a V-n...
56080,712915001,SPEED Frosty dress,265,Dress,Garment Full body,Lace,Dark Blue,Dark,Blue,Ladieswear,Jersey Fancy,"Short, sleeveless dress in lace with a round n..."
